In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.utils import filtrar_meses, verificar_columnas_y_tipos, estandarizar_nombres_columnas
from src.utils import funciones 
from src.utils import verificar_consistencia_coordenadas

In [ ]:
import pandas as pd

from utils import corregir_longitudes_inconsistentes

def procesar_recorridos_anio(ruta_csv_entrada, ruta_csv_salida, meses_a_eliminar, col_origen="fecha_origen_recorrido", col_destino="fecha_destino_recorrido"):

    df = pd.read_csv(ruta_csv_entrada)

    df_filtrado, filas_eliminadas_origen = filtrar_meses(df, col_origen, meses_a_eliminar)
    print(f"Se eliminaron {filas_eliminadas_origen} filas por fecha de origen.")

    df_filtrado, filas_eliminadas_destino = filtrar_meses(df_filtrado, col_destino, meses_a_eliminar)
    print(f"Se eliminaron {filas_eliminadas_destino} filas por fecha de destino.")

    total_eliminadas = filas_eliminadas_origen + filas_eliminadas_destino

    df_filtrado.to_csv(ruta_csv_salida, index=False)

    return df_filtrado, total_eliminadas

def limpiar_y_guardar_recorridos():
    años_columnas_a_dropear = {
        2020: [],
        2021: ["Género"],
        2022: ["X"],
        2023: []
    }

    for año, columnas in años_columnas_a_dropear.items():
        ruta_entrada = f"data/recorridos/raw/trips_{año}.csv"
        ruta_salida = f"data/recorridos/processed/trips_{año}_pr.csv"
        
        df = pd.read_csv(ruta_entrada, index_col=0)
        
        if columnas:
            df = df.drop(columns=[col for col in columnas if col in df.columns])
        
        df = df.reset_index(drop=True)
        df.to_csv(ruta_salida, index=False)


def forzar_tipos(df, columnas, tipo):
    for col in columnas:
        if col in df.columns:
            df[col] = df[col].astype(tipo)
    return df

def limpiar_y_normalizar_archivos(archivos, funciones):
    anios = [2020, 2021, 2022, 2023]

    estandarizar_nombres_columnas(archivos)

    for anio in [2020, 2023]:
        ruta = f"data/recorridos/processed/trips_{anio}_pr.csv"
        df = pd.read_csv(ruta)
        df = df.dropna()
        df.to_csv(ruta, index=False)

    for anio in anios:
        print(f"Procesando año {anio}...")
        ruta = f"data/recorridos/processed/trips_{anio}_pr.csv"
        df = pd.read_csv(ruta)
        df_limpio = funciones[str(anio)](df)
        df_limpio = forzar_tipos(df_limpio, ["id_usuario"], 'float64')
        df_limpio.to_csv(ruta, index=False)

    verificar_columnas_y_tipos(archivos)

def corregir_coordenadas_con_base_2024(path_2024, archivos_a_corregir):
    df_2024 = pd.read_csv(path_2024)
    coords_2024 = {}

    for rol in ["origen", "destino"]:
        id_col = f"id_estacion_{rol}"
        lat_col = f"lat_estacion_{rol}"
        lon_col = f"long_estacion_{rol}"

        for _, row in df_2024.iterrows():
            est_id = row[id_col]
            lat = row[lat_col]
            lon = row[lon_col]
            if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                coords_2024[est_id] = (lat, lon)

    for anio, path in archivos_a_corregir.items():
        if not os.path.exists(path):
            print(f"⚠️ No se encontró el archivo de {anio}")
            continue

        df = pd.read_csv(path)

        for rol in ["origen", "destino"]:
            id_col = f"id_estacion_{rol}"
            lat_col = f"lat_estacion_{rol}"
            lon_col = f"long_estacion_{rol}"

            def corregir_coord(row):
                est_id = row[id_col]
                if est_id in coords_2024:
                    lat, lon = coords_2024[est_id]
                    row[lat_col] = lat
                    row[lon_col] = lon
                return row

            df = df.apply(corregir_coord, axis=1)

        df.to_csv(path, index=False)
        print(f"✅ Archivo actualizado y sobrescrito: {path}")

from src.utils import corregir_latitud_estacion\

def corregir_estaciones_2020():
    path_2020 = "data/recorridos/processed/trips_2020_pr.csv"
    df_2020 = pd.read_csv(path_2020)
    df_2020 = corregir_latitud_estacion(df_2020, est_id=240)
    df_2020.to_csv(path_2020, index=False)
    corregir_longitudes_inconsistentes(path_2020)

def verificar_coordenadas_todos_los_anios(anios):
    for anio in anios:
        path = f"data/recorridos/processed/trips_{anio}_pr.csv"
        verificar_consistencia_coordenadas(path, anio=anio)

# SUPER FUNC

In [ ]:
ruta_entrada = "data/recorridos/raw/trips_2024.csv"
ruta_salida = "data/recorridos/processed/trips_2024_pr.csv"
meses = [9, 10, 11, 12]

df_final, total = procesar_recorridos_anio(ruta_entrada, ruta_salida, meses)

limpiar_y_guardar_recorridos()

archivos = [
    "data/recorridos/processed/trips_2020_pr.csv",
    "data/recorridos/processed/trips_2021_pr.csv",
    "data/recorridos/processed/trips_2022_pr.csv",
    "data/recorridos/processed/trips_2023_pr.csv",
    "data/recorridos/processed/trips_2024_pr.csv",
]

limpiar_y_normalizar_archivos(archivos, funciones)

archivos_corregir = {
    2020: "data/recorridos/processed/trips_2020_pr.csv",
    2021: "data/recorridos/processed/trips_2021_pr.csv",
    2022: "data/recorridos/processed/trips_2022_pr.csv",
    2023: "data/recorridos/processed/trips_2023_pr.csv",
}

corregir_coordenadas_con_base_2024("data/recorridos/processed/trips_2024_pr.csv", archivos_corregir)
from src.utils import corregir_longitudes_inconsistentes
corregir_longitudes_inconsistentes("data/recorridos/processed/trips_2021_pr.csv")
corregir_estaciones_2020()
verificar_coordenadas_todos_los_anios([2020, 2021, 2022, 2023])


# DATOS RANDOM

In [ ]:
def analizar_presencia_estaciones(archivos, anios):
    estaciones_por_anio = {}

    for archivo, anio in zip(archivos, anios):
        if not os.path.exists(archivo):
            print(f"⚠️ Archivo no encontrado para el año {anio}")
            continue

        df = pd.read_csv(archivo)
        estaciones_origen = df['id_estacion_origen'].dropna().unique()
        estaciones_destino = df['id_estacion_destino'].dropna().unique()
        estaciones = set(estaciones_origen).union(set(estaciones_destino))
        estaciones_por_anio[anio] = estaciones

    todas_las_estaciones = sorted(set.union(*estaciones_por_anio.values()))
    tabla_presencia = pd.DataFrame(index=todas_las_estaciones)

    for anio in anios:
        estaciones = estaciones_por_anio.get(anio, set())
        tabla_presencia[anio] = ["✓" if est in estaciones else "" for est in tabla_presencia.index]

    tabla_presencia["Anios_presente"] = tabla_presencia[anios].apply(lambda row: sum(cell == "✓" for cell in row), axis=1)
    tabla_presencia = tabla_presencia.sort_index()

    print("\n📋 Presencia de estaciones por año (por ID, ordenadas por cantidad de años presentes):")
    print(tabla_presencia)

    print("\n📈 Cantidad de estaciones por año:")
    for anio in anios:
        cantidad = len(estaciones_por_anio.get(anio, set()))
        print(f"  - {anio}: {cantidad} estaciones")

    return tabla_presencia, estaciones_por_anio


def mostrar_estaciones_faltantes_en_2024(estaciones_por_anio):
    print("\n🔍 Estaciones que aparecían en años anteriores pero NO en 2024:")

    estaciones_2024 = estaciones_por_anio.get(2024, set())

    for anio in [2020, 2021, 2022, 2023]:
        estaciones_anio = estaciones_por_anio.get(anio, set())
        solo_en_anio = sorted(estaciones_anio - estaciones_2024)
        print(f"\n➡️ Estaciones en {anio} pero NO en 2024 ({len(solo_en_anio)}):")
        print(solo_en_anio)

In [ ]:
archivos = [
    "data/recorridos/processed/trips_2020_pr.csv",
    "data/recorridos/processed/trips_2021_pr.csv",
    "data/recorridos/processed/trips_2022_pr.csv",
    "data/recorridos/processed/trips_2023_pr.csv",
    "data/recorridos/processed/trips_2024_pr.csv",
]
anios = [2020, 2021, 2022, 2023, 2024]

tabla, estaciones_por_anio = analizar_presencia_estaciones(archivos, anios)
mostrar_estaciones_faltantes_en_2024(estaciones_por_anio)

verificar_consistencia_coordenadas("data/recorridos/processed/trips_2024_pr.csv", anio=2024)